# Language Translation with `RNNs` and `Transformer` Models 

Return to the [castle](https://github.com/Nkluge-correa/teeny-tiny_castle).

**Language translation is the process of converting a text written in one language (the source language) into another language (the target language). In the context of machine learning (ML), language translation typically refers to the use of ML algorithms and models to automate this process.**

**There are several key challenges in building an effective ML-based language translation system. One is the need to handle the variability and complexity of natural language, which can involve ambiguous or context-dependent meanings, idioms, and other factors that can make translation difficult.** 

**Another challenge is the need to handle the vast number of possible language pairs, as well as the need to handle multiple target languages for a given source language. Despite these challenges, ML-based language translation systems have made significant progress in recent years and are now used in a variety of applications, including the translation of websites, documents, and spoken language in real time.**

**One of the biggest and most current advances in language modeling and _sequence-to-sequence_ machine translation was made possible by the invention of the `transformer` architecture.**

<img src="https://machinelearningmastery.com/wp-content/uploads/2021/08/attention_research_1.png" alt="drawing" height="450"/>

**A `transformer` model is a type of neural network architecture first described in the 2017 paper "_[Attention Is All You Need](https://arxiv.org/abs/1706.03762)_" by Vaswani et al. Transformers were originally used for machine translation, but this adaptable architecture is now used in a wide range of fields and problems.**

**In contrast to conventional convolutional neural networks (`CNNs`) or recurrent neural networks (`RNNs`), the `transformer` model uses `self-attention` mechanisms to dynamically weigh the input features and compute a weighted sum of the input features. In the context of natural language processing, where the meaning of a word can depend on the context of the other words in the sentence, this enables the model to capture long-range dependencies in the input data more effectively.**

**This notebook will show sequence-to-sequence modeling on a machine translation task, which is exactly the task for which the `Transformer` was developed. For compassion, we'll develop a recurrent sequence model (`GRU`) and the full `Transformer` architecture.**

**We’ll be working with an `English-to-Portuguese` translation dataset available [here](https://www.kaggle.com/datasets/nageshsingh/englishportuguese-translation). The downloaded text file contains one example per line: an English sentence, followed by a tab character, followed by the corresponding Portuguese sentence.**

**Let us load our dataset and print the first 10 samples. We also need to prepare the samples of our target language with special tokes (`[start]` and `[end]`), to help the generative part of our models to understand _when a sentence starts and ends_.**

In [7]:

with open("data/eng-por.txt", encoding='utf-8') as f:
    lines = f.read().split("\n")[:-1]

text_pairs = []
for line in lines: 
    english, portuguese, _ = line.split("\t")
    portuguese = "[start] " + portuguese + " [end]"
    text_pairs.append((english, portuguese))
display(text_pairs[: 10])

[('Go.', '[start] Vai. [end]'),
 ('Go.', '[start] Vá. [end]'),
 ('Hi.', '[start] Oi. [end]'),
 ('Run!', '[start] Corre! [end]'),
 ('Run!', '[start] Corra! [end]'),
 ('Run!', '[start] Corram! [end]'),
 ('Run.', '[start] Corre! [end]'),
 ('Run.', '[start] Corra! [end]'),
 ('Run.', '[start] Corram! [end]'),
 ('Who?', '[start] Quem? [end]')]

**Now, let's divide our datset them into the customary training, validation, and test sets after shuffling them.**

In [8]:
import random

random.shuffle(text_pairs)

num_val_samples = int(0.15 * len(text_pairs))

num_train_samples = len(text_pairs) - 2 * num_val_samples

train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

**The learned vocabulary for each language needs to be separated into two separate TextVectorization layers, which need to be created now (English and Portuguese). The "`[start]`" and "`[end]`" tokens that we have inserted must also be kept. The characters "`[ ]`" would normally be stripped, but we want to leave them in.**

**So that we can reuse the vocab learned, we will save them as `txt` files. In these files are all the words learned by the `TextVectorization` class.**

**Both vocabularies will have a maximum of 20,000 words, and all sequences will be truncated after 20 tokens/words.**

In [10]:
from keras import layers
import tensorflow as tf
import string
import re


strip_chars = string.punctuation
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, f"[{re.escape(strip_chars)}]", "")

vocab_size = 20000
sequence_length = 20

source_vectorization = tf.keras.layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length
    )

target_vectorization = tf.keras.layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
    )

train_english_texts = [pair[0] for pair in train_pairs]
train_portuguese_texts = [pair[1] for pair in train_pairs]

source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_portuguese_texts)

portuguese_vocab = target_vectorization.get_vocabulary()

with open(r'models/portuguese_vocabulary.txt', 'w', encoding='utf-8') as fp:
    for word in portuguese_vocab:
        fp.write("%s\n" % word)
    fp.close()

english_vocab = source_vectorization.get_vocabulary()

with open(r'models/english_vocabulary.txt', 'w', encoding='utf-8') as fp:
    for word in english_vocab:
        fp.write("%s\n" % word)
    fp.close()

**Now, we will create a `data pipeline` using our data. The dataset will be transformed into a "`(inputs, target)`" of `batch_size = 32` where the target is the Portuguese sentence offset by one step ahead (the next token in the input sequence). Meanwhile, inputs are a dictionary with two keys, "encoder inputs" (the English sentence) and "decoder inputs" (the Portuguese sentence).**

In [11]:
batch_size = 32

def format_dataset(eng, por):
    eng = source_vectorization(eng)
    por = target_vectorization(por)
    return ({
    "english": eng,
    "portuguese": por[:, :-1],
    }, por[:, 1:])

def make_dataset(pairs):
    eng_texts, por_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    por_texts = list(por_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, por_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(42).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

**Now we can train our first model, which will be an `RNN`. `Recurrent neural networks` dominated `sequence-to-sequence` learning from 2015–2017 before being surpassed by the Transformer. They were the basis for many real-world machine-translation systems, like the [Google Translate circa 2017](https://en.wikipedia.org/wiki/Google_Neural_Machine_Translation), which was powered by a stack of eight large `LSTM` layers.**

**But for this notebook, we will use [`GRUs`](https://en.wikipedia.org/wiki/Gated_recurrent_unit) instead of an [`LSTM`](https://en.wikipedia.org/wiki/Long_short-term_memory). The GRU is like an `LSTM` with a forget gate, but has fewer parameters than LSTM, as it lacks an output gate.**

**If you wish, you can stack more `GRUs` layers in the encoder and decoder parts. As it is, this model has already more than 40M parameters. Stacking recurrent layers will slow training down. Given the _sequence processing nature_ of `RNNs`, there are limits two how much we can parallelize.**

**We are given each learned word in our vocabulary an embedding dimension of 256. This is a dense vector that represents the "_relationships of a word with another_." The dense internal layer of our network will have 1024 nodes per `GRU`.**

> **Note: An `embedding` layer is a neural network layer that maps categorical variables, such as words in a vocabulary, into a continuous vector space. The goal of the embedding layer is to represent each word or categorical variable in a way that captures its relevant semantic and syntactic properties.**

> **Note: The `latent dimension` of a language model refers to the underlying vector space that represents the learned semantic and syntactic features of a language. In other words, it is the number of hidden variables used to represent the language model's internal state.**

In [15]:
from tensorflow import keras
from keras import layers

embed_dim = 256
latent_dim = 1024

source = keras.Input(shape=(None,), dtype="int64", name="english") 
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source) 
encoded_source = layers.Bidirectional(layers.GRU(latent_dim), merge_mode="sum")(x)

past_target = keras.Input(shape=(None,), dtype="int64", name="portuguese") 
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target) 
decoder_gru = layers.GRU(latent_dim, return_sequences=True)
x = decoder_gru(x, initial_state=encoded_source) 
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x) 
seq2seq_rnn = keras.Model([source, past_target], target_next_step)

seq2seq_rnn.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])

seq2seq_rnn.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 english (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 portuguese (InputLayer)        [(None, None)]       0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, None, 256)    5120000     ['english[0][0]']                
                                                                                                  
 embedding_3 (Embedding)        (None, None, 256)    5120000     ['portuguese[0][0]']             
                                                                                            

**Now all that is left to do is to train our `seq2seq_rnn`. Be mindful that without a GPU, the training of this model could take many hours.**

**If you wish to skip training and use the pre-trained model (`GRU_eng_por.keras`), you can download it in this [link](https://drive.google.com/uc?export=download&id=1q2S7xdKdsRCg5PSthvxNnUTuzo8gYUnx).**

In [16]:
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

callbacks = [keras.callbacks.ModelCheckpoint("models/GRU_eng_por.keras",
                                                save_best_only=True),
            keras.callbacks.EarlyStopping(monitor="val_loss",
                                            patience=10,
                                            verbose=1,
                                            mode="auto",
                                            baseline=None,
                                            restore_best_weights=True)]

seq2seq_rnn.fit(train_ds, epochs=15, validation_data=val_ds, callbacks=callbacks)

Version:  2.10.1
Eager mode:  True
GPU is available
Epoch 1/15
3695/3695 [==============================] - 1138s 305ms/step - loss: 1.3829 - accuracy: 0.5019 - val_loss: 1.1052 - val_accuracy: 0.5909
Epoch 2/15
3695/3695 [==============================] - 1138s 308ms/step - loss: 1.1060 - accuracy: 0.5992 - val_loss: 1.0455 - val_accuracy: 0.6304
Epoch 3/15
3695/3695 [==============================] - 1137s 308ms/step - loss: 1.0689 - accuracy: 0.6287 - val_loss: 1.0486 - val_accuracy: 0.6432
Epoch 4/15
3695/3695 [==============================] - 1143s 309ms/step - loss: 1.0639 - accuracy: 0.6431 - val_loss: 1.0517 - val_accuracy: 0.6495
Epoch 5/15
3695/3695 [==============================] - 1137s 308ms/step - loss: 1.0624 - accuracy: 0.6504 - val_loss: 1.0569 - val_accuracy: 0.6523
Epoch 6/15
3695/3695 [==============================] - 1131s 306ms/step - loss: 1.0613 - accuracy: 0.6543 - val_loss: 1.0570 - val_accuracy: 0.6548
Epoch 7/15
3695/3695 [==============================] 

**Accuracy is a crude way to monitor validation-set performance during this kind of task. On average, this model predicts the next word in the Portuguese sentence correctly $65\%$ of the time. However, in practice, next-token accuracy isn’t a great metric for machine translation models. During inference, you’re generating the target sentence from scratch, and you can’t rely on previously generated tokens (a.k.a. $100\%$ correctness does not mean you have a good translator). In real-world machine translation applications, we would likely use “_BLEU scores_” to evaluate our models.**

**`BLEU` (_bilingual evaluation understudy_) is an algorithm for evaluating the quality of text which has been machine-translated from one natural language to another. Quality is considered to be the correspondence between a machine's output and that of a human: "_the closer a machine translation is to a professional human translation, the better it is_" – this is the central idea behind BLEU.**

**To test our model, we can use sentences from our test set, or write some English sentences ourselves. Given an English sentence, we will feed the decoder block a `[start]` token, together with the encoded version of the English sentence, and the decoder will auto-regressively generate the next token, append to the decoded sequence (`[[start], [new token], ...]`), repeatedly, until the `[end]` token is generated, or the model reaches gets stomped by our maximum sentence length.** 

**Bellow, we import our trained model and vocabularies and test it on some sample text. The trained (or downloaded) model should be in the `models` folder.**



In [12]:
import tensorflow as tf
import numpy as np
import string
import re

strip_chars = string.punctuation
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")


def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, f"[{re.escape(strip_chars)}]", "")

seq2seq_rnn = tf.keras.models.load_model("models/GRU_eng_por.keras")

with open('models/portuguese_vocabulary.txt', encoding='utf-8',  errors='backslashreplace') as fp:
    portuguese_vocab = [line.strip() for line in fp]
    fp.close()

with open('models/english_vocabulary.txt', encoding='utf-8',  errors='backslashreplace') as fp:
    english_vocab = [line.strip() for line in fp]
    fp.close()


target_vectorization = tf.keras.layers.TextVectorization(max_tokens=20000,
                                        output_mode="int",
                                        output_sequence_length=21,
                                        standardize=custom_standardization,
                                        vocabulary=portuguese_vocab)

source_vectorization = tf.keras.layers.TextVectorization(max_tokens=20000,
                                        output_mode="int",
                                        output_sequence_length=20,
                                        vocabulary=english_vocab)

portuguese_index_lookup = dict(zip(range(len(portuguese_vocab)), portuguese_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"

    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])
        next_token_predictions = seq2seq_rnn.predict([tokenized_input_sentence, tokenized_target_sentence], verbose=0) 
        sampled_token_index = np.argmax(next_token_predictions[0, i, :]) 
        sampled_token = portuguese_index_lookup[sampled_token_index] 
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]": 
            break
    return decoded_sentence

eng_sentences =["What is its name?",
                "How old are you?",
                "I know you know where Mary is.",
                "We will show Tom.",
                "What do you all do?",
                "Don't do it!"]

for sentence in eng_sentences:
    print(f"English sentence:\n{sentence}")
    print(f'Portuguese transalation:\n{decode_sequence(sentence)}')
    print('-' * 50)

English sentence:
What is its name?
Portuguese transalation:
[start] qual é o nome [end]
--------------------------------------------------
English sentence:
How old are you?
Portuguese transalation:
[start] quantos anos você tem [end]
--------------------------------------------------
English sentence:
I know you know where Mary is.
Portuguese transalation:
[start] eu sei que você sabe onde maria está [end]
--------------------------------------------------
English sentence:
We will show Tom.
Portuguese transalation:
[start] nós vamos tom [end]
--------------------------------------------------
English sentence:
What do you all do?
Portuguese transalation:
[start] o que vocês faz [end]
--------------------------------------------------
English sentence:
Don't do it!
Portuguese transalation:
[start] não faça isso [end]
--------------------------------------------------


**As you can see, translations are far from perfect. To improve this model, we could:**

- **Use a deep stack of recurrent layers for both the encoder and the decoder.** 
- **We could use an `LSTM` instead of a `GRU`.**

**However, `RNNs` have limitations when it comes to sequence-to-sequence tasks and language modeling in general. For example,  due to their propensity to gradually forget the past, `RNNs` struggle to handle extremely long sequences. Consequently, `RNN`-based models are unable to retain long-term context, which is necessary for translating lengthy documents (which is one of the reasons why many online translation tools are limited by the size of the input).**

**It is because of these limitations that the machine learning community has embraced the `Transformer` architecture for sequence-to-sequence problems. And sequence-to-sequence learning is the task where Transformers excel the most.**

**Bellow, you can see the full sequence-to-sequence Transformer.**

![transformer-block](https://d2l.ai/_images/transformer.svg)

**_source_: [Dive into Deep Learning 11. Attention Mechanisms and Transformers - 11.7. The Transformer Architecture](https://d2l.ai/chapter_attention-mechanisms-and-transformers/transformer.html).**

**The transformer model consists of two main components: the `encoder` and the `decoder`.**

**The `encoder` is responsible for converting the input sequence into a set of internal representations that capture the relevant information in the input. It does this by applying a series of `self-attention` and `feedforward layers` to the input sequence. The `self-attention` layers allow the model to dynamically weigh the input features and compute a weighted sum of the features, which allows the model to capture long-range dependencies in the input data.**

**The `decoder` is responsible for generating the output sequence based on the internal representations produced by the `encoder`. It does this by applying a series of `self-attention` and f`eedforward layers` to the internal representations and a set of additional "_context_" vectors that are computed from the `encoder` output. The `decoder` also uses an `attention` mechanism to weigh the context vectors and incorporate information from the `encoder` output into the generation of the output sequence (a type of residual connection shared by both `transformer` blocks).**

**Together, the `encoder` and `decoder` allow the `transformer` model to effectively process and translate input sequences and capture long-range dependencies in the data.**

**For an extremely _comprehensive_ and _ilustrated_ explanation of what is "_attention_" or how a "_transformer works_", we recommend the work of _Jay Alammar_:**

- **[The Illustrated Transformer](https://jalammar.github.io/illustrated-transformer/).**
- **[The Illustrated GPT-2](https://jalammar.github.io/illustrated-gpt2/).**

**But the general "_gist_" of attention and self attention is this:**

**In a `transformer` model, `self-attention` is calculated using three input matrices: the query matrix ($Q$), the key matrix ($K$), and the value matrix ($V$). The `self-attention` mechanism computes a weighted sum of the value matrix ($V$) based on the similarity between the queries ($Q$) and the keys ($K$).**

**The query, key, and value matrices are typically derived from the input data by applying a linear transformation to the input features. The dimensions of the matrices depend on the size of the input and the number of attention "_heads_" used by the model.**

**To compute `self-attention`, the model first computes the dot product of the query and key matrices, which produces a matrix of dot products. The dot products are then divided by the square root of the dimensionality of the key matrix to ensure that the `self-attention` weights are well-behaved. Finally, the dot products are passed through a `softmax` function to produce a matrix of attention weights that sum to $1$.**

**The attention weights are then used to weight the value matrix ($V$) and compute a weighted sum of the values, which is then used as the output of the `self-attention` mechanism. This process is repeated for each attention head and the output of all the attention heads is concatenated to produce the final `self-attention` output.**

**The `self-attention` mechanism allows the model to dynamically weigh the input features and compute a weighted sum of the features based on the similarity between the queries and the keys. This allows the model to capture long-range dependencies in the input data and effectively process input sequences of variable length.**

**Here is the equation for self-attention calculation in transformer models:**

$$ Attention(Q, K, V) = softmax(\frac{QK^T}{\sqrt{d_k}})V $$

**where** 
- **$Q$ is the query matrix.**
- **$K$ is the key matrix.**
- **$V$ is the value matrix.**
- **$d_k$ is the dimensionality of the key matrix.**

## Transformer Encoder

**Fundamentally, the `encoder` part is a "_text classification_" machine. It's a very generic module that takes in a sequence and learns how to transform it into a more useful representation.**

**We used the `decoder` part to make a toxicity classifier in this [notebook](xxx). And [Ai.ra](https://github.com/Nkluge-correa/Aira-EXPERT), AIRES closed domain chatbot, was built (in its last interaction), as a `decoder-only transformer`.**

**The `decoder` block is made up of dense layers that process sequence tokens independently of one another, as well as an attention layer that examines the tokens as a group. You could rearrange the tokens in a sequence and still get the same pairwise attention scores.**

**`Self-attention` is a set-processing mechanism that focuses on the relationships between pairs of sequence elements; unlike `RNNs`, it is not concerned with the order of the tokens.**

**Below we implement the `encoder` block using the subclass features of the `Keras API`.**

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras import layers

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

## Positional Embedding

**As said before, `self-attention` is an order-agnostic technique. However, as done by [Vaswaniet al.](https://arxiv.org/abs/1706.03762), we can manually inject order information in the representations through _positional encoding_ (i.e., give the model access to word-order information).**

**The original "_Attention is all you need_" paper added to the word embeddings a vector containing values in the range [-1, 1] that varied cyclically depending on the position (a cosine function).**

**We’ll simply use the `tf.keras.layers.Embedding` layer to create a positional encoding parallel to the normal `Embedding` layer.**

In [3]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

## Transformer Decoder

**The `transformer decoder` is the model's second half. It reads tokens $0....N$ in the target sequence and tries to predict token $N+1$, just like the `RNN decoder`. While doing so, it employs neural attention to determine which tokens in the encoded source sentence are most closely related to the target token it is currently attempting to predict.**

**An extra bit of complication that the decoder brings is the idea of _causal padding_. Causal padding is critical for successfully training sequence-to-sequence Transformers.**

**Causal padding is used to ensure that the network does not violate the temporal or causal order of the data. Unlike an `RNN`, which looks at its input one step at a time and thus only has access to steps $0....N$ to generate output step $N+1$, the `TransformerDecoder` can look at the entire target sequence at once (without casual padding). If it could use all of its input, it would simply learn to copy input step $N+1$. As a result, the model would achieve perfect training accuracy while learning nothing useful. And that is why we need causal padding. To "_hide the future_" from out model during training.**

![causal-attention-padding](https://jalammar.github.io/images/gpt2/transformer-attention-mask.png)

**_Source:_ [The Illustrated GPT-2 (Visualizing Transformer Language Models)](https://jalammar.github.io/illustrated-gpt2/).**



In [4]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

**When we put all of the puzzle pieces together, we get the complete end-to-end Transformer. It simply combines the components we've created thus far: the `PositionalEmbedding` layers, the `TransformerEncoder`, and the `TransformerDecoder`. Similarly to our GRU model, we could stack these decoder and encoder blocks one on top of the other to create a more robust model.**

**However, let us keep it small, and use just one `encoder-decoder` block. The original Transformer model consists of 6 stacked `encoder-decoder` blocks**

In [5]:
embed_dim = 256
dense_dim = 2048
num_heads = 8
vocab_size = 20000
sequence_length = 20

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="portuguese")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x) 

transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])

transformer.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 english (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 portuguese (InputLayer)        [(None, None)]       0           []                               
                                                                                                  
 positional_embedding (Position  (None, None, 256)   5125120     ['english[0][0]']                
 alEmbedding)                                                                                     
                                                                                                  
 positional_embedding_1 (Positi  (None, None, 256)   5125120     ['portuguese[0][0]']         

**As you can see, this model is almost half the size of our RNN, and will be a little bit faster to train it! However, if you wish to skip training and use the pre-trained model (`transformer_eng_por.keras`), you can download it in this [link](https://drive.google.com/uc?export=download&id=1NTTDKZZ41Vw-3IVMB_jLstqlqPp77vqy).**

In [13]:
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

callbacks = [keras.callbacks.ModelCheckpoint("models/transformer_eng_por.keras",
                                                save_best_only=True),
            keras.callbacks.EarlyStopping(monitor="val_loss",
                                            patience=10,
                                            verbose=1,
                                            mode="auto",
                                            baseline=None,
                                            restore_best_weights=True)]

transformer.fit(train_ds, epochs=30, validation_data=val_ds, callbacks=callbacks)

Version:  2.10.1
Eager mode:  True
GPU is available
Epoch 1/30
3695/3695 [==============================] - 602s 162ms/step - loss: 1.4840 - accuracy: 0.4742 - val_loss: 1.1986 - val_accuracy: 0.5541
Epoch 2/30
3695/3695 [==============================] - 603s 163ms/step - loss: 1.1662 - accuracy: 0.5791 - val_loss: 1.0980 - val_accuracy: 0.6068
Epoch 3/30
3695/3695 [==============================] - 603s 163ms/step - loss: 1.1069 - accuracy: 0.6144 - val_loss: 1.0850 - val_accuracy: 0.6247
Epoch 4/30
3695/3695 [==============================] - 602s 163ms/step - loss: 1.0882 - accuracy: 0.6326 - val_loss: 1.0726 - val_accuracy: 0.6365
Epoch 5/30
3695/3695 [==============================] - 604s 163ms/step - loss: 1.0702 - accuracy: 0.6451 - val_loss: 1.0600 - val_accuracy: 0.6459
Epoch 6/30
3695/3695 [==============================] - 604s 163ms/step - loss: 1.0511 - accuracy: 0.6548 - val_loss: 1.0492 - val_accuracy: 0.6525
Epoch 7/30
3695/3695 [==============================] - 602s

**Bellow, we import our trained model and vocabularies and test it on some sample text. Since we are using Keras subclassess, we will need to load our models like this:**


```python

transformer = keras.models.load_model("transformer-eng-pot.keras", 
    custom_objects={"TransformerEncoder": TransformerEncoder, 
        "PositionalEmbedding": PositionalEmbedding,
        "TransformerDecoder": TransformerDecoder})
                                                 
```

**The trained (or downloaded) model should be in the `models` folder.**

In [14]:
import tensorflow as tf
import numpy as np
import string
import keras
import re

strip_chars = string.punctuation
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")


def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, f"[{re.escape(strip_chars)}]", "")

transformer = keras.models.load_model("models/transformer_eng_por.keras", 
    custom_objects={"TransformerEncoder": TransformerEncoder, 
        "PositionalEmbedding": PositionalEmbedding,
        "TransformerDecoder": TransformerDecoder})

with open('models/portuguese_vocabulary.txt', encoding='utf-8', errors='backslashreplace') as fp:
    portuguese_vocab = [line.strip() for line in fp]
    fp.close()

with open('models/english_vocabulary.txt', encoding='utf-8', errors='backslashreplace') as fp:
    english_vocab = [line.strip() for line in fp]
    fp.close()


target_vectorization = tf.keras.layers.TextVectorization(max_tokens=20000,
                                        output_mode="int",
                                        output_sequence_length=21,
                                        standardize=custom_standardization,
                                        vocabulary=portuguese_vocab)

source_vectorization = tf.keras.layers.TextVectorization(max_tokens=20000,
                                        output_mode="int",
                                        output_sequence_length=20,
                                        vocabulary=english_vocab)

portuguese_index_lookup = dict(zip(range(len(portuguese_vocab)), portuguese_vocab))
max_decoded_sentence_length = 20


def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"

    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = portuguese_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence


eng_sentences =["What is its name?",
                "How old are you?",
                "I know you know where Mary is.",
                "We will show Tom.",
                "What do you all do?",
                "Don't do it!"]

for sentence in eng_sentences:
    print(f"English sentence:\n{sentence}")
    print(f'Portuguese transalation:\n{decode_sequence(sentence)}')
    print('-' * 50)

English sentence:
What is its name?
Portuguese transalation:
[start] qual é o nome dele [end]
--------------------------------------------------
English sentence:
How old are you?
Portuguese transalation:
[start] quantos anos você tem [end]
--------------------------------------------------
English sentence:
I know you know where Mary is.
Portuguese transalation:
[start] eu sei que você sabe onde mary está [end]
--------------------------------------------------
English sentence:
We will show Tom.
Portuguese transalation:
[start] vamos ligar para o tom [end]
--------------------------------------------------
English sentence:
What do you all do?
Portuguese transalation:
[start] o que vocês todos nós têm feito [end]
--------------------------------------------------
English sentence:
Don't do it!
Portuguese transalation:
[start] não faça isso [end]
--------------------------------------------------


**You can now compare the output of both models, and the `transformer` (_being lighter and faster to train_) shows a better performance than the `RNN`. However, there is a problem with the translation of the first sentence.**

**While the `RNN` translates `What is its name?` to `[start] o que é o seu nome [end]`, the `transformer` model makes a gender assumption, even though the source sentence wasn’t gendered (`[start] qual é o nome dele [end]`). Errors like these are not uncommon in NLP, algorithmic bias being one of the great problems associated with the use of language models in real applications.**

**In AI literature, we call this a "_hallucination_." When a machine learning model "_hallucinates_" something, it means that the model is producing output that is not based on the input data and is not representative of the underlying distribution of the data. This can occur when the model is overfitting to the training data and has learned to memorize specific examples rather than generalizing to new data.**

**Hallucinations can occur in a variety of forms, depending on the type of ML model and the task it is trained to perform. For example, in the case of image classification, a model might hallucinate an object or pattern that is not present in the input image. In natural language processing, a model might generate nonsensical or unrelated text.**

**Hallucinations can be a sign of overfitting and can indicate that the model is not generalizing well to new data. To prevent hallucinations, it is important to use techniques such as regularization and early stopping to prevent overfitting and to train models with curated, high-quality, datasets to prevent the positioning of the model with unwanted biases.**

---

Return to the [castle](https://github.com/Nkluge-correa/teeny-tiny_castle).